## Dataset
- You will be using a modified version of the FairFace dataset (https://github.com/joojs/fairface). This is a set of 86,744 training face images and 10,954 validation face images. 
- In order to decrease the training time I converted all images to gray scale and resized them to 32 × 32. Each face has 3 different attributes which can be used for a classification task: race, gender, and age. All files can be found in the zip file on Canvas. The train folder contains the training images and the fairface label train.csv file contains all the label. There is a similar folder and file for the validation set.
- As the three different attributes have a different number of possible values, your final layers for each classifier will vary. For each of the networks below please attempt to classify 2 of the attributes (you can choose which).

## Imports

In [120]:
#Tensor imports
import tensorflow as tf
from tensorflow import optimizers
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.callbacks import TensorBoard

#Pillow Imports
from PIL import Image

#Import Pandas
import pandas as pd

#Import Numpy
import numpy as np

#Sci_Kit Imports
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix

#Import datetime
import datetime

## Load TensorBoard and Create Logs

In [121]:
%load_ext tensorboard
%reload_ext tensorboard
#log_folder = "logs"
log_folder = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


## Task 1: Fully Connected Neural Network
1. Build a feed forward neural network with the following specifications (Test on two different tasks):
    - Hidden layer 1: 1024 neurons with hyperbolic tangent activation function in each neuron.
    - Hidden layer 2: 512 neurons, with sigmoid activation function in each of the neuron.
    - 100 neurons, with rectified linear activation function in each of the neuron.
    - Output layer: n (depending on the task) neurons representing the n classes, using the softmax activation function.
2. Using Min-Max scaling to scale the training dataset and using the same Min and Max values from the training set scale the test dataset (X−Xmin/Xmax−Xmin).
3. Using mini-batch gradient descent to optimize the loss function: “categorical cross-entropy” on the training dataset. Please record the loss value for each of the epochs and create an epoch-loss plot and an accuracy-loss plot for both the training and validation set.
4. Report the following:
    - Final classification accuracy.
    - The n-class confusion matrix.

In [122]:
def createFeedFoward(inputShape, outputSize, lr):
    model = Sequential()
    model.add(layers.Dense(1024, input_shape=inputShape, activation='tanh'))
    model.add(layers.Dense(512, activation='sigmoid'))
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(outputSize, activation='softmax'))
    opt = optimizers.SGD(learning_rate=lr)
    model.compile(loss='CategoricalCrossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [123]:
def createXY(imgPath, labelFile, num):
    #Create and normalize X
    X = []
    for i in range(num):
        fileName = imgPath + str(i+1) + '.jpg'
        img = Image.open(fileName)
        X.append(list(img.getdata()))
    scaler = MinMaxScaler()
    scaler.fit(X)
    X = scaler.transform(X)
    
    #Get labels
    label_df = pd.read_csv(labelFile)
    
    #Find unique labels and output size
    age_labels = label_df['age'].unique()
    race_labels = label_df['race'].unique()
    
    #Create Binary y arrays
    lb_age = LabelBinarizer(sparse_output=False)
    lb_race = LabelBinarizer(sparse_output=False)
    lb_age.fit(age_labels)
    lb_race.fit(race_labels)
    y_age = list(label_df['age'].head(num))
    y_race = list(label_df['race'].head(num))
    y_age = lb_age.transform(y_age)
    y_race = lb_age.transform(y_race)
    
    
    return X, y_age, y_race, age_labels, race_labels 

In [124]:
def printResults(predictions, labels, trueLabels):
    print(len(predictions))
    print(len(predictions[1]))
    print(len(labels))
    print(len(trueLabels))
    for i in range(len(predictions)):
        print("Label: " + trueLabels[i])
        for j in range(len(labels)):
            print("{:12}: {:10.2f}%".format(labels[j], (predictions[i][j] * 100)))

In [125]:
def getMax(values):
    maxes = [np.argmax(val) for val in values ]
    return maxes

In [126]:
callbacks = [TensorBoard(log_dir=log_folder,
                         histogram_freq=1,
                         write_graph=True,
                         write_images=True,
                         update_freq='epoch',
                         profile_batch=2,
                         embeddings_freq=1)]

2023-04-06 19:22:59.680965: I tensorflow/core/profiler/lib/profiler_session.cc:101] Profiler session initializing.
2023-04-06 19:22:59.680988: I tensorflow/core/profiler/lib/profiler_session.cc:116] Profiler session started.
2023-04-06 19:22:59.682284: I tensorflow/core/profiler/lib/profiler_session.cc:128] Profiler session tear down.


In [127]:
X_train, y_age_train, y_race_train, age_labels, race_labels = createXY('project3_COSC525/train/', 'project3_COSC525/fairface_label_train.csv', 86744)
X_test, y_age_test, y_race_test, _ , _ = createXY('project3_COSC525/val/', 'project3_COSC525/fairface_label_val.csv', 10954)

In [128]:
def taskOne(X_train, y_train, X_test, y_test, lr, numEpochs, batchSize, port):
    model = createFeedFoward((1024,), len(y_train[0]), lr)
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=numEpochs, batch_size=batchSize, callbacks=callbacks)
    y_true = getMax(y_test)
    y_pred = getMax(model.predict(X_test))
    eval = tf.keras.metrics.Accuracy()
    eval.update_state(y_true, y_pred)
    print('Accuracy: ', eval.result().numpy())
    c_matrix = confusion_matrix(y_true, y_pred)
    print(c_matrix)
    %tensorboard --logdir logs --port=port

In [ ]:
taskOne(X_train, y_age_train, X_test, y_age_test, 0.001, 120, 100, 6009)

In [ ]:
taskOne(X_train, y_race_train, X_test, y_race_test, 0.001, 120, 100, 6009)

## Task 2: Small Convolutional Neural Network
- Build a convolutional neural network with the following specifications (Test on two different tasks):
    - Convolution layer having 40 feature detectors, with kernel size 5 x 5, and ReLU as the activation function, with stride 1 and no-padding.
    - A max-pooling layer with pool size 2x2.
    - Fully connected layer with 100 neurons, and ReLU as the activation function.
    - Output layer: n (depending on the task) neurons representing the n classes, using the softmax activation function. function for each of the 10 neurons.
2. Using Min-Max scaling to scale the training dataset and using the same Min and Max values from the training set scale the test dataset ( X−Xmin/Xmax−Xmin ).
3. Using mini-batch gradient descent to optimize the loss function: “categorical cross-entropy” on the training dataset. Please record the loss value for each of the epochs and create an epoch-loss plot and an accuracy-loss plot for both the training and validation set.
4. Report the following:
    - Final classification accuracy.
    - The n-class confusion matrix.

In [147]:
# Setup CNN network model
def createSmallCNN(inputShape, outputSize, lr):
    model = Sequential()
    model.add(layers.Conv2D(40, (5, 5), activation='relu', input_shape=inputShape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(outputSize, activation='softmax'))
    opt = optimizers.SGD(learning_rate=lr)
    model.compile(loss='CategoricalCrossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [148]:
# Get normalized data
X_train, y_age_train, y_race_train, age_labels, race_labels = createXY('project3_COSC525/train/', 'project3_COSC525/fairface_label_train.csv', 10000)
X_test, y_age_test, y_race_test, _ , _ = createXY('project3_COSC525/val/', 'project3_COSC525/fairface_label_val.csv', 1000)

In [149]:
# Perform task two; train and evaluate CNN for given train/test data
def taskTwo(X_train, y_train, X_test, y_test, lr, numEpochs, batchSize, port):
    model = createSmallCNN((32, 32, 1), len(y_train[0]), lr)
    model.fit(np.reshape(X_train, (X_train.shape[0], 32, 32)), y_train, validation_data=(np.reshape(X_test, (X_test.shape[0], 32, 32)), y_test), epochs=numEpochs, batch_size=batchSize, callbacks=callbacks)
    y_true = getMax(y_test)
    y_pred = getMax(model.predict(np.reshape(X_test, (X_test.shape[0], 32, 32))))
    eval = tf.keras.metrics.Accuracy()
    eval.update_state(y_true, y_pred)
    print('Accuracy: ', eval.result().numpy())
    c_matrix = confusion_matrix(y_true, y_pred)
    print(c_matrix)
    %tensorboard --logdir logs --port=port

In [ ]:
taskTwo(X_train, y_age_train, X_test, y_age_test, 0.001, 120, 100, 6009)

In [ ]:
taskTwo(X_train, y_race_train, X_test, y_race_test, 0.001, 120, 100, 6009)

## Task 3: Your own Convolutional Neural Network
1. Build another convolutional neural network, where you choose all the parameters to see if you can get a higher accuracy.
2. Using Min-Max scaling to scale the training dataset and using the same Min and Max values from the training set scale the test dataset ( X−Xmin/Xmax−Xmin ).
3. Using mini-batch gradient descent to optimize the loss function: “categorical cross-entropy” on the training dataset. Please record the loss value for each of the epochs and create an epoch-loss plot and an accuracy-loss plot for both the training and validation set.
4. Report the following:
    - Final classification accuracy.
    - The n-class confusion matrix

In [130]:
# Setup CNN network model
def createOwnCNN(inputShape, outputSize, lr):
    model = Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=inputShape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(outputSize, activation='softmax'))
    opt = optimizers.SGD(learning_rate=lr)
    model.compile(loss='CategoricalCrossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [140]:
# Get normalized data
X_train, y_age_train, y_race_train, age_labels, race_labels = createXY('project3_COSC525/train/', 'project3_COSC525/fairface_label_train.csv', 10000)
X_test, y_age_test, y_race_test, _ , _ = createXY('project3_COSC525/val/', 'project3_COSC525/fairface_label_val.csv', 1000)

In [141]:
# Perform task three; train and evaluate CNN for given train/test data
def taskThree(X_train, y_train, X_test, y_test, lr, numEpochs, batchSize, port):
    model = createOwnCNN((32, 32, 1), len(y_train[0]), lr)
    model.fit(np.reshape(X_train, (X_train.shape[0], 32, 32)), y_train, validation_data=(np.reshape(X_test, (X_test.shape[0], 32, 32)), y_test), epochs=numEpochs, batch_size=batchSize, callbacks=callbacks)
    y_true = getMax(y_test)
    y_pred = getMax(model.predict(np.reshape(X_test, (X_test.shape[0], 32, 32))))
    eval = tf.keras.metrics.Accuracy()
    eval.update_state(y_true, y_pred)
    print('Accuracy: ', eval.result().numpy())
    c_matrix = confusion_matrix(y_true, y_pred)
    print(c_matrix)
    %tensorboard --logdir logs --port=port


In [ ]:
taskThree(X_train, y_age_train, X_test, y_age_test, 0.001, 120, 100, 6009)

In [ ]:
taskThree(X_train, y_race_train, X_test, y_race_test, 0.001, 120, 100, 6009)

## Task 4: Your own Convolutional Neural Network on both Tasks Simultaneously
1. Build another convolutional neural network, where you try and classify both tasks with a single network. After your flatten layer have two more fully connected layers for each “branch”. Note that in order to do so you will not be able to use the Sequential model.
2. Using Min-Max scaling to scale the training dataset and using the same Min and Max values from the training set scale the test dataset ( X−Xmin/Xmax−Xmin ).
3. Using mini-batch gradient descent to optimize the loss function: “categorical cross-entropy” on the training dataset. Please record the loss value for each of the epochs and create an epoch-loss plot and an accuracy-loss plot for both the training and validation set.
4. Report the following:
    - Final classification accuracy.
    - The n-class confusion matrix

In [193]:
# Setup CNN network model
def createOwnCNNTwoTasks(inputShape, outputSizes, lr):
    input = layers.Input(shape = inputShape)
    conv1 = layers.Conv2D(32, (3, 3), activation='relu', name='conv1')(input)
    max1 = layers.MaxPooling2D((2, 2), name='max1')(conv1)
    conv2 = layers.Conv2D(64, (3, 3), activation='relu', name='conv2')(max1)
    max2 = layers.MaxPooling2D((2, 2), name='max2')(conv2)
    flatten = layers.Flatten()(max2)

    # Branch 1
    fc11 = layers.Dense(100, activation='relu', name='fc11')(flatten)
    fc12 = layers.Dense(outputSizes[0], activation='softmax', name='fc12')(fc11)

    # Branch 2
    fc21 = layers.Dense(100, activation='relu', name='fc21')(flatten)
    fc22 = layers.Dense(outputSizes[1], activation='softmax', name='fc22')(fc21)

    # Concatenate output of branches
    output = layers.concatenate([fc12, fc22])
    model = keras.Model(inputs=input, outputs=output)

    opt = optimizers.SGD(learning_rate=lr)
    model.compile(loss='CategoricalCrossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [194]:
# Get normalized data
X_train, y_age_train, y_race_train, age_labels, race_labels = createXY('project3_COSC525/train/', 'project3_COSC525/fairface_label_train.csv', 10000)
X_test, y_age_test, y_race_test, _ , _ = createXY('project3_COSC525/val/', 'project3_COSC525/fairface_label_val.csv', 1000)

In [195]:
# Perform task four; train and evaluate CNN for given train/test data
def taskFour(X_train, y_trains, X_test, y_tests, lr, numEpochs, batchSize, port):
    model = createOwnCNNTwoTasks((32, 32, 1), (len(y_trains[0][0]), len(y_trains[1][0])), lr)
    model.fit(np.reshape(X_train, (X_train.shape[0], 32, 32)), np.concatenate((y_trains[0], y_trains[1]), axis=1), validation_data=(np.reshape(X_test, (X_test.shape[0], 32, 32)), np.concatenate((y_tests[0], y_tests[1]), axis=1)), epochs=numEpochs, batch_size=batchSize, callbacks=callbacks)
    y_true = getMax(np.concatenate((y_tests[0], y_tests[1]), axis=1))
    y_pred = getMax(model.predict(np.reshape(X_test, (X_test.shape[0], 32, 32))))
    eval = tf.keras.metrics.Accuracy()
    eval.update_state(y_true, y_pred)
    print('Accuracy: ', eval.result().numpy())
    c_matrix = confusion_matrix(y_true, y_pred)
    print(c_matrix)
    %tensorboard --logdir logs --port=port

In [ ]:
taskFour(X_train, (y_age_train, y_race_train), X_test, (y_age_test, y_race_test), 0.001, 120, 100, 6009)

## Task 5: Variational Auto Encoder (COSC 525 only)
1. Build a variational autoencoder with the following specifications (in this one you have a little more flexibility):
    - Should have at least two convolution layers in the encoder and 2 deconvolution layers in the decoder.
    - Latent dimension should be at least 5.
    - Loss should be either MSE or binary cross entropy.
2. Using Min-Max scaling to scale the training dataset and using the same Min and Max values from the training set scale the test dataset ( X−Xmin/Xmax−Xmin ).
3. Using mini-batch gradient descent to optimize the loss function on the training dataset. Please record the loss value for each of the epochs and create an epoch-loss plot and an accuracy-loss plot for both the training and validation set.
4. Qualitatively evaluate your model by generating a set of faces by randomly choosing 10 latent vectors and presenting the resulting images

VAE code adapted from https://keras.io/examples/generative/vae/

In [85]:
# Sampling layer
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [86]:
# Encoder model
latent_dim = 15

encoder_inputs = keras.Input(shape=(32, 32, 1))
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(16, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 32, 32, 1)]  0           []                               
                                                                                                  
 conv2d_12 (Conv2D)             (None, 16, 16, 32)   320         ['input_9[0][0]']                
                                                                                                  
 conv2d_13 (Conv2D)             (None, 8, 8, 64)     18496       ['conv2d_12[0][0]']              
                                                                                                  
 conv2d_14 (Conv2D)             (None, 4, 4, 64)     36928       ['conv2d_13[0][0]']              
                                                                                            

In [87]:
# Decoder model
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(2 * 2 * 64, activation="relu")(latent_inputs)
x = layers.Reshape((2, 2, 64))(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2DTranspose(1, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 15)]              0         
                                                                 
 dense_9 (Dense)             (None, 256)               4096      
                                                                 
 reshape_4 (Reshape)         (None, 2, 2, 64)          0         
                                                                 
 conv2d_transpose_17 (Conv2D  (None, 4, 4, 64)         36928     
 Transpose)                                                      
                                                                 
 conv2d_transpose_18 (Conv2D  (None, 8, 8, 64)         36928     
 Transpose)                                                      
                                                                 
 conv2d_transpose_19 (Conv2D  (None, 16, 16, 64)       3692

In [88]:
# Variational AutoEncoder model class
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.mean_squared_error(data, reconstruction), axis=(1, 2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [ ]:
# Create and train VAE on all face images
X_train, _, _, _, _ = createXY('project3_COSC525/train/', 'project3_COSC525/fairface_label_train.csv', 'age', 10000)
X_test, _, _ , _, _ = createXY('project3_COSC525/val/', 'project3_COSC525/fairface_label_val.csv', 'age', 1000)
all_faces = np.concatenate([X_train, X_test], axis=0)

vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.SGD(learning_rate=0.0005))
vae.fit(np.reshape(all_faces, (11000, 32, 32, 1)), epochs=10000, batch_size=128, verbose=0)

In [78]:
# Test generating some (non-random) faces
preds = vae.encoder.predict(np.reshape(all_faces, (10, 32, 32, 1)))[0]
# gen = vae.decoder.predict(np.reshape(np.mean(preds, axis=0), (1, 5)))[0]
gen = vae.decoder.predict(np.reshape(preds[4], (1, latent_dim)))[0]
gen = np.reshape(gen, (32, 32))
im = Image.fromarray(np.uint8(gen * 255), 'L')
im.save("test.png")
comp_im = Image.open("project3_COSC525/train/5.jpg")
comp_im.save("test_comp.png")

1/1 [==============================] - 0s 24ms/step
